In [1]:
import numpy as np
import pandas as pd
import math
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as Image
import random
import collections
from skimage import measure, color
from skimage.feature import local_binary_pattern
from sklearn.cluster import KMeans

In [2]:
k_num = [2, 5, 6, 3, 3, 
         4, 2, 3, 3, 5, 
         5, 6, 5, 4, 5,
         3, 5, 5, 6, 6,
         7, 6, 5, 7, 7, 
         4, 5, 7, 7, 6, 
         7, 6, 7, 5, 6,
         7, 6, 7, 9, 6,
         2, 3, 7, 7, 6, 
         5, 6]

In [3]:
def IOU(in1, in2):
    
    img1 = in1.astype('int')
    img2 = in2.astype('int')
    I = int(np.sum(img1 & img2))
    U = int(np.sum(img1 | img2))
    out = round((I / U), 4)
    
    return out

In [4]:
def LoG(img, ksize, scale):
    
    temp = np.copy(img)
    gaussian = cv2.GaussianBlur(temp, (ksize, ksize), 0)
    out = cv2.Laplacian(gaussian, -1, ksize = ksize, scale = scale)
    
    return out

In [5]:
def gabor(img, ksize, scale, theta):
    
    temp = np.copy(img)
    kernel = cv2.getGaborKernel((ksize, ksize), scale, np.pi*theta/180, ksize/2, 1, 0, cv2.CV_32F)
    kernel /= math.sqrt((kernel * kernel).sum())
    out = cv2.filter2D(img, -1, kernel)
    
    return out

In [6]:
def integralHist(img, n = 11, w = 0):
    
    temp0 = np.copy(img).astype(int)
    temp0.resize([1, img.shape[0]*img.shape[1]])
    temp = 11 * (temp0[0] - temp0[0].min())
    width = float((temp.max() - temp.min()) / n)
    temp.resize([img.shape[0], img.shape[1]])
    
    def oneBin(src, z1, z2, w = 0):
        
        padding = w - 1
        
        RS = np.zeros(src.shape)
        HI = np.zeros(src.shape)
        for i in range(src.shape[0]):
            for j in range(src.shape[1]):
                if i == 0:
                    RS[i][j] = 1 if ((src[i][j] >= z1)&(src[i][j] < z2)) else 0
                else:
                    RS[i][j] = RS[i-1][j] + 1 if ((src[i][j] >= z1)&(src[i][j] < z2)) else RS[i-1][j]
                if j == 0:
                    HI[i][j] = 1 if ((src[i][j] >= z1)&(src[i][j] < z2)) else 0
                else:
                    HI[i][j] = HI[i][j-1] + RS[i][j]
        
        if w != 0:
            padding0 = np.zeros([HI.shape[0], padding])
            padding1 = np.zeros([padding, HI.shape[1] + padding])
            tempA = np.r_[np.c_[HI, padding0], padding1]
            tempB = np.r_[np.c_[padding0, HI], padding1]
            tempC = np.r_[padding1, np.c_[HI, padding0]]
            tempD = np.r_[padding1, np.c_[padding0, HI]]
            out = tempA + tempD - tempB - tempC
        else:
            out = HI
        
        out = out.astype(int)
#         dst = out[int(w/2):out.shape[0]-int(w/2), int(w/2):out.shape[1]-int(w/2)]
        dst = out[:out.shape[0]-w+1, :out.shape[1]-w+1]
        dst = dst.reshape([1, dst.shape[0]*dst.shape[1]])
        
        return dst[0].astype(int)
    
    bins = list()
    for k in range(0, n):
#         print('Initializing Bin', k, '...')
        HI = oneBin(temp, k*width, (k+1)*width, w)
#         print('Bin', k, 'finished')
        bins.append(HI)
    
    return bins

In [7]:
def cleaning(img):
    
    N = img.shape[0] * img.shape[1]
    clusters = int(img.max())
    temp = img.astype('int')
    out = np.copy(img)
    for i in range(0, clusters):
        seg = np.argwhere(temp == i)
        if (seg.shape[0] < 0.01 * N):
            if seg[:,0].min() != seg[:,0].max():
                p00 = seg[:,0].min()
                p01 = seg[:,0].max()
            else:
                p00 = seg[0,0]
                p01 = p00 + 1
            if seg[:,1].min() != seg[:,1].max():
                p10 = seg[:,1].min()
                p11 = seg[:,1].max()
            else:
                p10 = seg[0,1]
                p11 = p10 + 1
            window = temp[p00:p01, p10:p11]
            (value, counter) = np.unique(window, return_counts=True)
            count = counter
            count[np.argmax(counter)] = 0
            for p in range(seg.shape[0]):
                out[seg[p][0], seg[p][1]] = value[np.argmax(count)]
    
    return out

In [8]:
def segImg(img, clusters, kernel_size = 5):
    
    # integration scale
    temp = int(0.04 * np.max(img.shape))
    w_s = temp if temp%2 != 0 else temp - 1 
    window_size = w_s if w_s > 14 else 21
    
    # kernel size
    k_size = kernel_size
    
    # the L*a*b* color space is more perceptually uniform when intensity filter applied
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(img_lab)
    median_L = cv2.medianBlur(l, k_size)
    median_a = cv2.medianBlur(a, k_size)
    median_b = cv2.medianBlur(b, k_size)
    
    # gray scale image
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g0_25 = gabor(img_gray, window_size, 3, 0)
    g1_25 = gabor(img_gray, window_size, 3, 45)
    g2_25 = gabor(img_gray, window_size, 3, 90)
    g3_25 = gabor(img_gray, window_size, 3, 135)
    
    # feature extraction and splicing
    bins_L = integralHist(median_L, w = window_size)
    print('L channel finished')
    bins_A = integralHist(median_a, w = window_size)
    print('a channel finished')
    bins_B = integralHist(median_b, w = window_size)
    print('b channel finished')
    bins0 = integralHist(g0_25, w = window_size)
    print('Gabor-0 finished')
    bins1 = integralHist(g1_25, w = window_size)
    print('Gabor-45 finished')
    bins2 = integralHist(g2_25, w = window_size)
    print('Gabor-90 finished')
    bins3 = integralHist(g3_25, w = window_size)
    print('Gabor-135 finished')
    
    Y = np.concatenate((np.array(bins_L), np.array(bins_A), np.array(bins_B), 
                        np.array(bins0), np.array(bins1), np.array(bins2), np.array(bins3)),axis=0)
    
    # k-means clustering
    estimator = KMeans(n_clusters = clusters)
    estimator.fit(Y.T)
    label_pred = estimator.labels_
    centroids = estimator.cluster_centers_
    
    # segmentation
    w = window_size
    out = label_pred.reshape([img.shape[0], img.shape[1]])
    temp = out[int(w/2):, int(w/2):out.shape[1]]
    temp = np.ones(temp.shape) + temp.astype('int')
    labels = measure.label(temp, connectivity=2)
    x = labels.max() + 1
    padding = int(w/2)
    padding0 = x * np.ones([labels.shape[0], padding])
    padding1 = x * np.ones([padding, labels.shape[1] + padding])
    del(out)
    out = np.r_[np.c_[labels, padding0], padding1]
    out = out - np.ones(out.shape)
    del(temp)
    temp = out
    del(out)
    out = cleaning(temp)
    
    return out

In [9]:
def oneAxis(img, ellipse, angle):
    
    points = np.nonzero(img)
    N = len(points[0])
    x0 = ellipse[0][0]
    y0 = ellipse[0][1]
    
    theta = (angle)*np.pi/180
    temp = 2*np.tan(theta)/(np.power(np.tan(theta), 2) + 1)
    
    if round(np.tan(theta), 4) == 0.0:
        t = 0
        out = list()
        for i in range(N):
            x = points[1][i]
            y = points[0][i]
            X = x
            Y = int(round((2 * y0 - y), 0))
            if (Y < img.shape[0])&(X < img.shape[1]):
                if img[Y][X] != 0:
                    t = t + 1
                    out.append([y, x])
                    
    elif round(1/np.tan(theta), 4) == 0.0:
        t = 0
        out = list()
        for i in range(N):
            x = points[1][i]
            y = points[0][i]
            X = int(round((2 * x0 - x), 0))
            Y = y
            if (Y < img.shape[0])&(X < img.shape[1]):
                if img[Y][X] != 0:
                    t = t + 1
                    out.append([y, x])
                    
    else:
        t = 0
        out = list()
        for i in range(N):
            x = points[1][i]
            y = points[0][i]
            X = int(round((temp*(np.tan(theta)*x0 - y0 + x/np.tan(theta) + y) - x), 0))
            Y = int(round((temp*(y0/np.tan(theta) - x0 + x + np.tan(theta)*y) - y), 0))
            if (Y < img.shape[0])&(X < img.shape[1]):
                if img[Y][X] != 0:
                    t = t + 1
                    out.append([y, x])
    
    return (round(t/N, 4))

In [10]:
def center(img, ellipse):
    
    points = np.nonzero(img)
    N = len(points[0])
    x0 = ellipse[0][0]
    y0 = ellipse[0][1]
    
    t = 0
    out = list()
    for i in range(N):
        x = points[1][i]
        y = points[0][i]
        X = int(round(2*x0 - x, 0))
        Y = int(round(2*y0 - y, 0))
        if (Y < img.shape[0])&(X < img.shape[1]):
            if img[Y][X] != 0:
                t = t + 1
                out.append([y, x])
    
    return (round(t/N, 4))

In [11]:
def fitting(img):
    
    N = img.shape[0] * img.shape[1]
    clusters = int(img.max())
    temp = img.astype('int')
    E = list()
    for i in range(0, clusters):
        seg = np.zeros(img.shape)
        pos = np.argwhere(temp == i)
        for p in range(pos.shape[0]):
            seg[pos[p][0], pos[p][1]] = 255
        seg = seg.astype('uint8')
        cont, hier = cv2.findContours(seg, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        if len(cont) == 0:
            continue
        maxlen = max(len(cont[j]) for j in range(len(cont)))
        for i in range(len(cont)):
            if len(cont[i]) == maxlen:
                cnt = cont[i]
        if len(cnt) < 5:
            continue
        img_edge = np.zeros(seg.shape, seg.dtype)
        img_fit = np.zeros(seg.shape, seg.dtype)
        ellipse = cv2.fitEllipse(cnt)
        cv2.drawContours(img_edge, [cnt], 0, (1, 1, 1), -1)
        cv2.ellipse(img_fit, ellipse, (1, 1, 1), -1)
        
        # area limitation
        if (len(img_fit.nonzero()[0]) < 0.01 * N):
            continue
        if (len(img_fit.nonzero()[0]) > N):
            continue
        if (len(img_fit.nonzero()[0]) < (3/4*((ellipse[1][0]/2)*(ellipse[1][1]/2)*np.pi))):
            continue
        
        # the similarity between the area selected to an ellipse
        if (IOU(img_edge, img_fit) < 0.8):
            continue
        if (ellipse[1][0] > img.shape[0])|(ellipse[1][1] > img.shape[1]):
            continue
        
        # the aspect ratio
        if(ellipse[1][0] > ellipse[1][1]):
            if ellipse[1][1]/ellipse[1][0] > 0.9:
                ratio = 1
            else: 
                ratio = 1 if (ellipse[2] < 35)|(ellipse[2] > 145) else 0
        else:
            if ellipse[1][0]/ellipse[1][1] > 0.9:
                ratio = 1
            else:
                ratio = 1 if (ellipse[2] > 55)&(ellipse[2] < 125) else 0
        if ratio == 0:
            continue
        
        E.append(ellipse)
    
    if len(E) > 1:
        EG = list()
        for i in range(len(E)):
            ground = np.zeros(img.shape)
            cv2.ellipse(ground, E[i], (1,1,1), -1)
            EG.append(ground)
        
        temp_del = list()
        for x in range(len(EG)-1):
            for y in range(x+1, len(EG)):
                if IOU(EG[x], EG[y]) > 0.95:
                    temp_del.append(y)
                    
        delete = list()
        [delete.append(i) for i in temp_del if not i in delete]
        out = list()
        for i in range(len(E)):
            if i in delete:
                continue
            out.append(E[i])
    else:
        out = E
        
    return out

In [12]:
# i = 10
# I = str(i)
# print('Pic', i, 'is processing:\n')
# img = cv2.imread(str(i) + ".jpg")
# gt = cv2.imread("G" + str(i) + ".jpg", 0)
# seg = segImg(img, k_num[i])
# # E = fitting(seg)
# plt.imshow(seg)

In [13]:
# E = fitting(seg)
# print('In picture', i, len(E), 'ellipses detected.')
# img_fit = np.zeros(seg.shape)
# img_merge = np.copy(img)
# for k in range(len(E)):
#     cv2.ellipse(img_fit, E[k], (255, 255, 255), -1)
#     cv2.ellipse(img_merge, E[k], (0, 255, 0), -1)
# plt.imshow(img_fit)

In [14]:
# k_size = 5
# img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
# l, a, b = cv2.split(img_lab)
# median_L = cv2.medianBlur(l, k_size)
# median_a = cv2.medianBlur(a, k_size)
# median_b = cv2.medianBlur(b, k_size)

# temp = int(0.04 * np.max(img.shape))
# w_s = temp if temp%2 != 0 else temp - 1 
# window_size = w_s if w_s > 14 else 21
# # gray scale image
# img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# g0_25 = gabor(img_gray, window_size, 3, 0)
# g1_25 = gabor(img_gray, window_size, 3, 45)
# g2_25 = gabor(img_gray, window_size, 3, 90)
# g3_25 = gabor(img_gray, window_size, 3, 135)

In [15]:
# fig = plt.figure(figsize=(15,15))

# ax = plt.subplot(331)
# plt.imshow(median_L)

# ax = plt.subplot(332)
# plt.imshow(median_a)

# ax = plt.subplot(333)
# plt.imshow(median_b)

# ax = plt.subplot(334)
# plt.imshow(g0_25)

# ax = plt.subplot(335)
# plt.imshow(g1_25)

# ax = plt.subplot(336)
# plt.imshow(g2_25)

# ax = plt.subplot(337)
# plt.imshow(g3_25)

In [16]:
Segment = list()
Fitting = list()
Merge = list()
GT = list()

for i in range(47):
    I = str(i)
    print('Pic', i, 'is processing:\n')
    img = cv2.imread(str(i) + ".jpg")
    gt = cv2.imread("G" + str(i) + ".jpg", 0)
    GT.append(gt)
    seg = segImg(img, k_num[i])
    Segment.append(seg)
    E = fitting(seg)
    print('In picture', i, len(E), 'ellipses detected.')
    img_fit = np.zeros(seg.shape)
    img_merge = np.copy(img)
    for k in range(len(E)):
        cv2.ellipse(img_fit, E[k], (255, 255, 255), -1)
        cv2.ellipse(img_merge, E[k], (0, 255, 0), -1)
    Fitting.append(img_fit)
    Merge.append(img_merge)
    out_show = np.expand_dims(img_fit, axis=2)
    out_show = np.concatenate((out_show, out_show, out_show), axis=-1)
    out_show = cv2.normalize(out_show, out_show, 0, 255, cv2.NORM_MINMAX)
    out_img = np.concatenate((img, out_show), axis = 1)
    out_img = out_img.astype('uint8')
    file_name = './out/Seg/Gabor/segmentation/S' + I + '.png'
    Image.imsave(file_name, seg)
    file_name = './out/Seg/Gabor/compare/C' + I + '.png'
    Image.imsave(file_name, out_img)
    file_name = './out/Seg/Gabor/merge/M' + I + '.png'
    Image.imsave(file_name, img_merge)
    print('\nPic', i, 'is done!\n')

Pic 0 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 0 1 ellipses detected.

Pic 0 is done!

Pic 1 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 1 2 ellipses detected.

Pic 1 is done!

Pic 2 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 2 0 ellipses detected.

Pic 2 is done!

Pic 3 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 3 1 ellipses detected.

Pic 3 is done!

Pic 4 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 4 0 ellipses detected.

Pic 4 

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 40 1 ellipses detected.

Pic 40 is done!

Pic 41 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 41 1 ellipses detected.

Pic 41 is done!

Pic 42 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 42 2 ellipses detected.

Pic 42 is done!

Pic 43 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 43 0 ellipses detected.

Pic 43 is done!

Pic 44 is processing:

L channel finished
a channel finished
b channel finished
Gabor-0 finished
Gabor-45 finished
Gabor-90 finished
Gabor-135 finished
In picture 44 1 ellipses detected.

Pic 44 is done!

In [17]:
exact = list()
almost = list()
none = list()
less = list()
more = list()
wrong = list()
for i in range(47):
    gt = GT[i]
    res = Fitting[i]
    value = IOU(gt, res)
    if value == 0.0:
        if (len(np.nonzero(res)[0]) == 0):
            none.append(i)
            print(i, 'none:', value)
        else:
            wrong.append(i)
            print(i, 'wrong:', value)
    elif value >= 0.85:
        exact.append(i)
        print(i, 'exact:', value)
    elif (value < 0.85)&(value > 0.7):
        almost.append(i)
        print(i, 'almost:', value)
    else:
        if (len(np.nonzero(res)[0]) > len(np.nonzero(gt)[0])):
            more.append(i)
            print(i, 'more:', value)
        else:
            less.append(i)
            print(i, 'less:', value)
print('\nNone', len(none), '\nLess:', len(less), '\nWrong', len(wrong), '\nMore', len(more), '\nAlmost', len(almost), '\nExact', len(exact))

0 exact: 0.9671
1 more: 0.5238
2 none: 0.0
3 almost: 0.8486
4 none: 0.0
5 wrong: 0.0
6 exact: 0.9674
7 almost: 0.762
8 wrong: 0.0
9 more: 0.4492
10 less: 0.2611
11 less: 0.1978
12 exact: 0.8765
13 wrong: 0.0
14 exact: 0.891
15 almost: 0.7967
16 none: 0.0
17 none: 0.0
18 more: 0.1366
19 none: 0.0
20 more: 0.2541
21 none: 0.0
22 wrong: 0.0
23 less: 0.6675
24 more: 0.1703
25 none: 0.0
26 none: 0.0
27 none: 0.0
28 more: 0.1282
29 wrong: 0.0
30 wrong: 0.0
31 none: 0.0
32 none: 0.0
33 none: 0.0
34 wrong: 0.0
35 wrong: 0.0
36 none: 0.0
37 none: 0.0
38 wrong: 0.0
39 wrong: 0.0
40 exact: 0.9347
41 exact: 0.9648
42 almost: 0.7419
43 none: 0.0
44 exact: 0.9247
45 exact: 0.95
46 more: 0.2986

None 15 
Less: 3 
Wrong 10 
More 7 
Almost 4 
Exact 8


In [18]:
more

[1, 9, 18, 20, 24, 28, 46]

In [23]:
def ellipseSet(img):
    
    temp = np.copy(img)
    ret,temp = cv2.threshold(temp,127,255,cv2.THRESH_BINARY)
    labels = measure.label(temp, connectivity=2)
    del(temp)
    num = labels.max()
    
    ELL = list()
    for i in range(num):
        temp = labels - (i+1)*np.ones(labels.shape)
        points = np.nonzero(temp)
        tempImg = np.zeros(labels.shape)
        for j in range(len(points[0])):
            tempImg[points[0][j]][points[1][j]] = 1
        tempImg = tempImg.astype('uint8')
        cont, hier = cv2.findContours(tempImg, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        maxlen = max(len(cont[k]) for k in range(len(cont)))
        for p in range(len(cont)):
            if len(cont[p]) == maxlen:
                cnt = cont[p]
        ellipse = cv2.fitEllipse(cnt)
        ELL.append(ellipse)
    
    return ELL

In [19]:
def evalE(numList):
    
    set_GT = list()
    set_FIT = list()
    for i in range(47):
        set_GT.append(ellipseSet(GT[i]))
        set_FIT.append(ellipseSet(Fitting[i]))
        
    # evaluation according to ellipses
    tp = 0
    tn = 0
    fp = 0
    fn = 0

    for i in numList:
        if len(set_FIT[i]) == 0:
            fn += len(set_GT[i])
        else:
            gtImg = list()
            for j in range(len(set_GT[i])):
                tempImg = np.zeros(Segment[i].shape)
                cv2.ellipse(tempImg, set_GT[i][j], (1,1,1), -1)
                gtImg.append(tempImg)
            for j in range(len(set_FIT[i])):
                temp = np.zeros(Segment[i].shape)
                cv2.ellipse(temp, set_FIT[i][j], (1,1,1), -1)
                value_set = list()
                for k in range(len(gtImg)):
                    value_set.append(IOU(temp, gtImg[k]))
                value_array = np.array(value_set)
                if max(value_array) < 0.7:
                    fp += 1
                else:
                    tp += 1
                for k in range(len(value_set)):
                    if value_array[k] > 0.7:
                        del gtImg[k]
            fn += len(gtImg)
    
    total = tp + tn + fp + fn

    print('Recall:', tp/(tp+fn), 
          '\nPrecision:', tp/(tp+fp), 
          '\nAccuracy:', (tp+tn)/total, 
          '\nF1:', 2*(tp)/(2*tp+fp+fn))

In [20]:
def evalP(numList):
    
    # evaluation according to pixels
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in numList:
        gt = (GT[i] > 127).astype(int)
        res = (Fitting[i]/255).astype(int)
        gt_ = (gt == 0).astype(int)
        res_ = (res == 0).astype(int)
        a = np.ones(gt.shape).astype(int)
        total = gt.shape[0]*gt.shape[1]
        temp_tp = sum(sum(res&gt))
        temp_tn = sum(sum(a - (res|gt)))
        temp_fp = sum(sum(res - (res&gt)))
        temp_fn = sum(sum(gt - (res&gt)))
        TP += temp_tp
        FP += temp_fp
        TN += temp_tn
        FN += temp_fn
    TOTAL = TP + TN + FP + FN

    rec = TP/(TP+FN)
    pre = TP/(TP+FP)
    acc = (TP+TN)/TOTAL
    f1 = 2*pre*rec/(pre+rec)
    print('Recall:', rec, '\nPrecision:', pre, '\nAccuracy:', acc, '\nF1:', f1)

In [21]:
L = list()
M = list()
S = list()
XS = list()

for i in range(47):
    nums = len(np.nonzero(GT[i])[0])
    sums = GT[i].shape[0]*GT[i].shape[1]
#     print(i, round(nums/sums, 4))
    if nums/sums >= 0.2:
        L.append(i)
    elif nums/sums >= 0.08:
        M.append(i)
    elif nums/sums >= 0.01:
        S.append(i)
    else:
        XS.append(i)

len(L)+len(M)+len(S)+len(XS)

47

In [24]:
print('In large group, the numbers of images are:\n', L)

print('\nThe results of evaluation according to ellipses are:')
evalE(L)

print('\nThe results of evaluation according to pixels are:')
evalP(L)

In large group, the numbers of images are:
 [0, 3, 10, 13, 14, 15, 16, 46]

The results of evaluation according to ellipses are:
Recall: 0.4444444444444444 
Precision: 0.4444444444444444 
Accuracy: 0.2857142857142857 
F1: 0.4444444444444444

The results of evaluation according to pixels are:
Recall: 0.4235559364908246 
Precision: 0.5411663118298741 
Accuracy: 0.7899241238857774 
F1: 0.47519211753099566


In [25]:
print('In medium group, the numbers of images are:\n', M)

print('\nThe results of evaluation according to ellipses are:')
evalE(M)

print('\nThe results of evaluation according to pixels are:')
evalP(M)

In medium group, the numbers of images are:
 [1, 2, 4, 5, 6, 7, 8, 9, 11, 12, 40, 41]

The results of evaluation according to ellipses are:
Recall: 0.46153846153846156 
Precision: 0.46153846153846156 
Accuracy: 0.3 
F1: 0.46153846153846156

The results of evaluation according to pixels are:
Recall: 0.8205720901451184 
Precision: 0.8913564345916082 
Accuracy: 0.9689691111954405 
F1: 0.8545008766760518


In [26]:
print('In small group, the numbers of images are:\n', S)

print('\nThe results of evaluation according to ellipses are:')
evalE(S)

print('\nThe results of evaluation according to pixels are:')
evalP(S)

In small group, the numbers of images are:
 [18, 19, 20, 22, 23, 24, 26, 27, 28, 34, 42, 43, 44, 45]

The results of evaluation according to ellipses are:
Recall: 0.35714285714285715 
Precision: 0.29411764705882354 
Accuracy: 0.19230769230769232 
F1: 0.3225806451612903

The results of evaluation according to pixels are:
Recall: 0.44444788575354344 
Precision: 0.15084269743449588 
Accuracy: 0.9342027642666176 
F1: 0.2252403106969563


In [27]:
print('In extra small group, the numbers of images are:\n', XS)

print('\nThe results of evaluation according to ellipses are:')
evalE(XS)

print('\nThe results of evaluation according to pixels are:')
evalP(XS)

In extra small group, the numbers of images are:
 [17, 21, 25, 29, 30, 31, 32, 33, 35, 36, 37, 38, 39]

The results of evaluation according to ellipses are:
Recall: 0.0 
Precision: 0.0 
Accuracy: 0.0 
F1: 0.0

The results of evaluation according to pixels are:
Recall: 0.0 
Precision: 0.0 
Accuracy: 0.9639144515831024 
F1: nan


/Users/nathan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: RuntimeWarning: invalid value encountered in double_scalars
